In [2]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import pickle
from autosklearn.classification import AutoSklearnClassifier

In [3]:
base = np.load(f"../extract_segments_bins/bins/bins_50.npy")

afib = base[base[:, -1] == 1]
afib = afib[np.random.choice(afib.shape[0], 5000, replace=False)]  # Comment this line

snr = base[base[:, -1] == -1]
snr = snr[np.random.choice(snr.shape[0], 5000, replace=False)]  # Comment this line
snr[:, -1] = np.zeros((snr.shape[0],))

base_ready = np.vstack((afib, snr))

y = base_ready[:, -1]
X = base_ready[:, 0:-1]

In [4]:
include = {
    "classifier": ["mlp", "libsvm_svc", "liblinear_svc"],
    "feature_preprocessor": ["no_preprocessing"],
}

automl = AutoSklearnClassifier(
    time_left_for_this_task=10800,
    per_run_time_limit=100,
    tmp_folder=f"/home/renan/autosklearn_classification_example/",
    n_jobs=-1,
    resampling_strategy="cv",
    include=include,
)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

automl.fit(X_train, y_train)

[WARNING] [2023-06-06 20:59:12,319:Client-AutoMLSMBO(1)::1b63f757-04c6-11ee-b199-f389fb898965] Configuration 532 not found
[WARNING] [2023-06-06 20:59:12,320:Client-AutoMLSMBO(1)::1b63f757-04c6-11ee-b199-f389fb898965] Configuration 1 not found
[WARNING] [2023-06-06 20:59:12,320:Client-AutoMLSMBO(1)::1b63f757-04c6-11ee-b199-f389fb898965] Configuration 454 not found
[WARNING] [2023-06-06 20:59:12,320:Client-AutoMLSMBO(1)::1b63f757-04c6-11ee-b199-f389fb898965] Configuration 230 not found
[WARNING] [2023-06-06 20:59:12,320:Client-AutoMLSMBO(1)::1b63f757-04c6-11ee-b199-f389fb898965] Configuration 138 not found
[WARNING] [2023-06-06 20:59:12,320:Client-AutoMLSMBO(1)::1b63f757-04c6-11ee-b199-f389fb898965] Configuration 506 not found
[WARNING] [2023-06-06 20:59:12,320:Client-AutoMLSMBO(1)::1b63f757-04c6-11ee-b199-f389fb898965] Configuration 473 not found
[WARNING] [2023-06-06 20:59:12,320:Client-AutoMLSMBO(1)::1b63f757-04c6-11ee-b199-f389fb898965] Configuration 585 not found
[WARNING] [2023-06

In [32]:
print(automl.leaderboard())
predictions = automl.predict(X_test)
print("Accuracy score:", accuracy_score(y_test, predictions))

with open("automl.pkl", "wb") as f:
    pickle.dump(automl, f)

          rank  ensemble_weight type      cost   duration
model_id                                                 
839          1             0.02  mlp  0.031000  74.634849
769          2             0.02  mlp  0.031143  93.939597
248          3             0.02  mlp  0.031143  91.303134
253          5             0.16  mlp  0.031286  81.195418
734          6             0.02  mlp  0.031286  80.889832
1325         4             0.14  mlp  0.031286  90.203340
726          7             0.04  mlp  0.031286  74.327182
473          8             0.02  mlp  0.031286  92.962405
1592         9             0.04  mlp  0.031286  93.652849
1174        10             0.06  mlp  0.031429  75.361478
755         11             0.04  mlp  0.031429  93.782395
881         12             0.04  mlp  0.031429  75.813030
1012        13             0.04  mlp  0.031429  87.838580
1264        15             0.06  mlp  0.031429  65.332708
1389        14             0.04  mlp  0.031429  83.860303
239         17

In [ ]:
with open("automl.pkl", "rb") as f:
    automl = pickle.load(f)
automl.leaderboard()

In [35]:
model = automl.show_models().get(248)["estimators"][0]["sklearn_classifier"]
print(model)

MLPClassifier(alpha=4.452045861848066e-07, beta_1=0.999, beta_2=0.9,
              hidden_layer_sizes=(67,),
              learning_rate_init=0.0003137392105124784, max_iter=256,
              n_iter_no_change=32, random_state=1, validation_fraction=0.0,
              verbose=0, warm_start=True)


: 